In [41]:
#Import packages

#Rdkit
from rdkit import Chem

#Scipy
from scipy import stats 
from scipy.stats import uniform,ranksums 

#Pandas
import pandas as pd
from pandas import DataFrame

#Others
from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler

import joblib 
import random
import numpy as np
import os
import math
import warnings

import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from plotnine import *

In [ ]:
#AlvaDesc (If 'AlvaDesc' is not installed, load the corresponding csv file)
from alvadesccliwrapper.alvadesc import AlvaDesc
#AlvaDesc Path(Linux default alvaDescCLI location)
aDesc = AlvaDesc('./') 

In [42]:
def print_RxC(dataframe):
    rows= dataframe.shape[0]
    columns= dataframe.shape[1]
    rows_columns=print(dataframe.shape[0],"Moleculars ×",dataframe.shape[1],"Descriptors")
    return rows_columns
def trimm_correlated(df_in, threshold):
    df_corr = df_in.corr(method='spearman')
    df_not_correlated = ~(df_corr.mask(np.tril(np.ones([len(df_corr)]*2, dtype=bool))).abs() > threshold).any()
    un_corr_idx = df_not_correlated.loc[df_not_correlated[df_not_correlated.index] == True].index
    df_out = df_in[un_corr_idx]
    return df_out

In [43]:
#Set the Work Path
warnings.filterwarnings ('ignore')
os.chdir("./")

In [44]:
#Read the data
Raw_data= pd.read_csv('./Original data/Raw_data.csv',index_col=0)
Raw_data['Hydrogel-forming ability']=np.where(Raw_data['Hydrogel-forming ability']=='Gelator', 1, 0)
Chemdraw_smiles=Raw_data['Chemdraw_smiles'].tolist()


In [45]:
Smis=[]
for i in Chemdraw_smiles:
    Smis.append(i)
Mols = []
for Smi in Smis:
    Mol = Chem.MolFromSmiles(Smi)
    Mols.append(Mol)
Canonical_smis = []
for Mol in Mols:
    Canonical_smi = Chem.MolToSmiles(Mol)
    Canonical_smis.append(Canonical_smi)
Raw_data['Canonical_smiles']=Canonical_smis
print_RxC(Raw_data)
Raw_data.head()

71 Moleculars × 3 Descriptors


,Hydrogel-forming ability,Chemdraw_smiles,Canonical_smiles
ID,,,
Ma_2019_A,1,OC[C@@H]1[C@H]([C@H]([C@H](N2C=NC3=C(N=CN=C32)...,Nc1ncnc2c1ncn2[C@@H]1O[C@H](CO)[C@@H](O)[C@H]1O
Ma_2019_U,1,O[C@H]1[C@@H](O)[C@H](N(C(N2)=O)C=CC2=O)O[C@@H...,O=c1ccn([C@@H]2O[C@H](CO)[C@@H](O)[C@H]2O)c(=O...
Ma_2019_C,1,O[C@H]1[C@@H](O)[C@H](N(C=CC(N)=N2)C2=O)O[C@@H...,Nc1ccn([C@@H]2O[C@H](CO)[C@@H](O)[C@H]2O)c(=O)n1
Ma_2019_G,1,O=C1C2=C(N=C(N1)N)N([C@H]3[C@H](O)[C@@H]([C@H]...,Nc1nc2c(ncn2[C@@H]2O[C@H](CO)[C@@H](O)[C@H]2O)...
Ma_2019_dA,1,O[C@H]1C[C@H](N(C=N2)C3=C2C(N)=NC=N3)O[C@@H]1CO,Nc1ncnc2c1ncn2[C@H]1C[C@H](O)[C@@H](CO)O1


In [46]:
Raw_data.to_csv("./Original data/Smiles_data.csv",sep=',',header=1,index=1)

In [47]:
#Calculate the descriotors
aDesc.set_input_SMILES(Canonical_smis)
if not aDesc.calculate_descriptors('ALL'):
 print('Error: ' + aDesc.get_error())
else:
 Data_gelation=aDesc.get_output()

In [48]:
X_original_data=pd.DataFrame(Data_gelation)
Desc_names_names=aDesc.get_output_descriptors()
X_original_data.columns=Desc_names_names
X_original_data.index=Raw_data.index
X_original_data.to_csv("./Original data/X_original_data.csv",sep=',',header=1,index=1)
print_RxC(X_original_data)
X_original_data.head()

71 Moleculars × 5666 Descriptors


,MW,AMW,Sv,Se,Sp,Si,Mv,Me,Mp,Mi,...,s1_numAroBonds,s2_numAroBonds,s3_numAroBonds,s4_numAroBonds,s34_size,s34_relSize,s34_phSize,s34_phRelSize,chiralMoment,chiralPhMoment
ID,,,,,,,,,,,,,,,,,,,,,
Ma_2019_A,267.28,8.352500,20.0734,33.3526,19.8921,36.9899,0.627294,1.042269,0.621628,1.155934,...,0.0,0.0,0.0,10.0,16.0,0.842105,7.75,0.407895,17.966397,8.618182
Ma_2019_U,244.23,8.421724,17.9656,30.5854,17.5454,33.3290,0.619503,1.054669,0.605014,1.149276,...,0.0,0.0,0.0,0.0,14.0,0.823529,6.75,0.397059,14.808251,7.026700
Ma_2019_C,243.25,8.108333,18.2722,31.3599,18.0966,34.6179,0.609073,1.045330,0.603220,1.153930,...,0.0,0.0,0.0,0.0,14.0,0.823529,6.75,0.397059,14.808251,7.026700
Ma_2019_G,283.28,8.584242,20.7882,34.6799,20.3466,38.1993,0.629945,1.050906,0.616564,1.157555,...,0.0,0.0,0.0,5.0,17.0,0.850000,8.75,0.437500,19.586399,10.257197
Ma_2019_dA,251.28,8.105806,19.3586,32.0253,19.4376,35.7805,0.624471,1.033074,0.627019,1.154210,...,0.0,0.0,0.0,10.0,15.0,0.833333,7.00,0.388889,17.259745,8.115820


In [49]:
# If 'AlvaDesc' is not installed, load the 'X_original_data.csv' file
X_original_data= pd.read_csv("./Original data/X_original_data.csv",header=0,index_col=0)
print_RxC(X_original_data)
X_original_data.head()

71 Moleculars × 5666 Descriptors


,MW,AMW,Sv,Se,Sp,Si,Mv,Me,Mp,Mi,...,s1_numAroBonds,s2_numAroBonds,s3_numAroBonds,s4_numAroBonds,s34_size,s34_relSize,s34_phSize,s34_phRelSize,chiralMoment,chiralPhMoment
ID,,,,,,,,,,,,,,,,,,,,,
Ma_2019_A,267.28,8.352500,20.0734,33.3526,19.8921,36.9899,0.627294,1.042269,0.621628,1.155934,...,0.0,0.0,0.0,10.0,16.0,0.842105,7.75,0.407895,17.966397,8.618182
Ma_2019_U,244.23,8.421724,17.9656,30.5854,17.5454,33.3290,0.619503,1.054669,0.605014,1.149276,...,0.0,0.0,0.0,0.0,14.0,0.823529,6.75,0.397059,14.808251,7.026700
Ma_2019_C,243.25,8.108333,18.2722,31.3599,18.0966,34.6179,0.609073,1.045330,0.603220,1.153930,...,0.0,0.0,0.0,0.0,14.0,0.823529,6.75,0.397059,14.808251,7.026700
Ma_2019_G,283.28,8.584242,20.7882,34.6799,20.3466,38.1993,0.629945,1.050906,0.616564,1.157555,...,0.0,0.0,0.0,5.0,17.0,0.850000,8.75,0.437500,19.586399,10.257197
Ma_2019_dA,251.28,8.105806,19.3586,32.0253,19.4376,35.7805,0.624471,1.033074,0.627019,1.154210,...,0.0,0.0,0.0,10.0,15.0,0.833333,7.00,0.388889,17.259745,8.115820


In [50]:
#Remove descriptors with excessive NA values
Thresh =int(0.8*len(X_original_data))
X_NAomit_data=X_original_data.dropna(
    #thresh=Thresh, 
    axis=1) 
print_RxC(X_NAomit_data)
X_NAomit_data.head()
X_NAomit_data.to_csv("./Original data/X_NAomit_data.csv",sep=',',header=1,index=1)

71 Moleculars × 4175 Descriptors


In [51]:
#Ranksum test
Ranksums_data=X_NAomit_data
Ranksums_data["Hydrogel-forming ability"]=Raw_data['Hydrogel-forming ability']
Notgelable=Ranksums_data.loc[Ranksums_data["Hydrogel-forming ability"] == 0]
Gelable=Ranksums_data.loc[Ranksums_data["Hydrogel-forming ability"] == 1]
Ranksums_result=[]
for i in list(Ranksums_data.columns):
 NG = Notgelable[i]
 G = Gelable[i]
 Ranksums_caculate=ranksums(NG, G, alternative='two-sided', nan_policy='omit')
 Ranksums_result.append(Ranksums_caculate)
Ranksums_results = pd.DataFrame(Ranksums_result, columns=['Statistics','Pvalues'])
Ranksums_results.index=Ranksums_data.columns#
Ranksums_results = Ranksums_results.drop('Hydrogel-forming ability',axis = 0)
Ranksums_results.sort_values(by="Pvalues" , inplace=True, ascending=True)
Ranksums_results.head()

,Statistics,Pvalues
MATS3p,3.758315,0.000171
MATS3v,3.677615,0.000235
SM10_AEA(dm),3.608444,0.000308
Eig01_EA(ed),3.608444,0.000308
SpMax_EA(ed),3.608444,0.000308


In [52]:
#Ranksum test <0.05
Ranksums_marked = Ranksums_results[Ranksums_results['Pvalues'] <0.05]
print(Ranksums_marked.shape)
Ranksums_marked.head()

(144, 2)


,Statistics,Pvalues
MATS3p,3.758315,0.000171
MATS3v,3.677615,0.000235
SM10_AEA(dm),3.608444,0.000308
Eig01_EA(ed),3.608444,0.000308
SpMax_EA(ed),3.608444,0.000308


In [53]:
X_ranksums_data=X_NAomit_data[Ranksums_marked.index]
print_RxC(X_ranksums_data)
X_ranksums_data.head()

71 Moleculars × 144 Descriptors


,MATS3p,MATS3v,SM10_AEA(dm),Eig01_EA(ed),SpMax_EA(ed),GATS3v,GATS7s,F07[N-O],SM12_EA(ed),SM09_EA(ed),...,minaaN,nLevel6,s2_pathLength,SpMAD_B(p),SpMAD_A,SM15_EA(ri),GNar,SM03_EA(ed),X0A,GATS8i
ID,,,,,,,,,,,,,,,,,,,,,
Ma_2019_A,-0.085445,-0.073032,11.607041,11.607041,11.607041,1.022785,0.962089,4.0,29.609609,22.312589,...,3.810499,1.500000,1.500000,1.492166,1.296386,16.951973,2.050213,7.548029,0.714134,0.981436
Ma_2019_U,-0.087509,-0.063423,11.152723,11.152723,11.152723,1.016541,1.131327,1.0,29.014596,21.800414,...,0.000000,0.750000,1.500000,1.439809,1.248566,16.540255,1.927525,7.264730,0.739823,0.587369
Ma_2019_C,-0.100433,-0.070546,11.152723,11.152723,11.152723,1.031143,1.023020,2.0,29.014596,21.800414,...,0.000000,0.750000,1.500000,1.429846,1.248566,16.497968,1.927525,7.264730,0.739823,0.913354
Ma_2019_G,-0.047845,-0.041945,11.639705,11.639705,11.639705,0.983313,0.940430,4.0,29.649090,22.344302,...,3.880152,1.750000,1.500000,1.461212,1.269881,17.118899,2.018429,7.603399,0.721940,0.995781
Ma_2019_dA,-0.061137,-0.051132,11.234674,11.234674,11.234674,1.008666,1.124941,2.0,29.134103,21.926368,...,3.853973,1.333333,1.666667,1.500617,1.304420,16.620737,2.086115,7.368340,0.705461,1.029772


In [56]:
#Scale data
Scaler = preprocessing.MinMaxScaler() #StandardScaler
Transformer =Scaler.fit(X_ranksums_data)
X_scaled_data=Transformer.transform(X_ranksums_data)
X_scaled_data =pd.DataFrame(X_scaled_data)
X_scaled_data.columns=X_ranksums_data.columns
X_scaled_data.index=Raw_data.index
X_scaled_data.to_csv("./Original data/X_scaled_data.csv",sep=',',header=1,index=1)
joblib.dump(Transformer, './Models/Scaler_transformer.pkl')
print_RxC(X_scaled_data)
X_scaled_data.head()

71 Moleculars × 144 Descriptors


,MATS3p,MATS3v,SM10_AEA(dm),Eig01_EA(ed),SpMax_EA(ed),GATS3v,GATS7s,F07[N-O],SM12_EA(ed),SM09_EA(ed),...,minaaN,nLevel6,s2_pathLength,SpMAD_B(p),SpMAD_A,SM15_EA(ri),GNar,SM03_EA(ed),X0A,GATS8i
ID,,,,,,,,,,,,,,,,,,,,,
Ma_2019_A,0.223067,0.344851,0.382490,0.382490,0.382490,0.689326,0.390779,0.500000,0.464641,0.497602,...,0.919692,0.363636,0.090909,0.674747,0.722310,0.523719,0.585611,0.552864,0.442403,0.586479
Ma_2019_U,0.218409,0.381663,0.270850,0.270850,0.270850,0.671199,0.589845,0.000000,0.317977,0.329816,...,0.000000,0.090909,0.090909,0.373885,0.326999,0.367153,0.174645,0.336752,0.830666,0.000000
Ma_2019_C,0.189242,0.354375,0.270850,0.270850,0.270850,0.713595,0.462449,0.166667,0.317977,0.329816,...,0.000000,0.090909,0.090909,0.316630,0.326999,0.351072,0.174645,0.336752,0.830666,0.485155
Ma_2019_G,0.307924,0.463942,0.390516,0.390516,0.390516,0.574721,0.365302,0.500000,0.474373,0.507991,...,0.936503,0.454545,0.090909,0.496875,0.503205,0.587196,0.479147,0.595103,0.560377,0.607829
Ma_2019_dA,0.277926,0.428750,0.290988,0.290988,0.290988,0.648332,0.582334,0.166667,0.347434,0.371078,...,0.930185,0.303030,0.151515,0.723311,0.788722,0.397757,0.705872,0.415790,0.311320,0.658417


In [55]:
#Remove high correlation descriptors
X_cor_data=trimm_correlated(X_scaled_data,0.80)
X_cor_data.index=Raw_data.index
X_cor_data.to_csv("./Original data/ML_data.csv",sep=',',header=1,index=1)
print_RxC(X_cor_data)
X_cor_data.head()

71 Moleculars × 40 Descriptors


,MATS3p,SM10_AEA(dm),GATS7s,F07[N-O],VE1sign_Dz(v),VE3sign_D/Dt,P_VSA_charge_4,CATS2D_09_DA,B09[O-O],CATS2D_06_DL,...,MATS2m,MATS2p,GATS6i,P_VSA_charge_2,Eig03_AEA(ed),CATS2D_05_DA,GATS2p,C-016,s2_pathLength,GATS8i
ID,,,,,,,,,,,,,,,,,,,,,
Ma_2019_A,0.223067,0.382490,0.390779,0.500000,0.339202,0.433674,0.000000,0.4,0.0,0.00,...,0.136148,0.791478,0.070041,0.015761,0.580423,0.500000,0.304895,0.0,0.090909,0.586479
Ma_2019_U,0.218409,0.270850,0.589845,0.000000,0.097243,0.770077,0.000000,0.2,1.0,0.25,...,0.028067,0.316380,0.138782,0.384100,0.200950,0.500000,0.711020,1.0,0.090909,0.000000
Ma_2019_C,0.189242,0.270850,0.462449,0.166667,0.081578,0.770077,0.359389,0.2,0.0,0.25,...,0.157809,0.441757,0.327739,0.025930,0.200950,0.500000,0.667729,1.0,0.090909,0.485155
Ma_2019_G,0.307924,0.390516,0.365302,0.500000,0.259153,0.508232,0.359389,0.6,1.0,0.00,...,0.224020,0.631157,0.262950,0.373931,0.587700,0.666667,0.484050,0.0,0.090909,0.607829
Ma_2019_dA,0.277926,0.290988,0.582334,0.166667,0.404032,0.457316,0.000000,0.4,0.0,0.25,...,0.230461,0.794139,0.231586,0.015761,0.536421,0.000000,0.242784,0.0,0.151515,0.658417
